# M2 | Submission: Wage Regression

Course: Gen Bus 656: Foundations of Statistical Learning for Business Analytics\
Author: Matthew Brown\
Date: October 4, 2025

## Submission Instructions

1. In the course GitHub repository, locate the file `'GB886_II_9_Wages_1985_Current_Population_Survey.csv'`. This data set is a cross-sectional sample from the May 1985 Current Population Survey by the US Census Bureau. 
2. These data include (hourly) wages for **534 individuals**, where we have information on: 
    + age
    + sex (0 for male, 1 for female)
    + race (H for Hispanic, W for White, O for Other)
    + years of education
    + et cetera
3. Set up a Codebook that loads this datafile.
4. Run a linear regression model that uses `Wage' as the outcome/target/dependent variable and all the available features/independent variables as variables (you will have to convert the categorical variables into dummy variables).
5. Produce a regression table to include:
    + R-squared
    + RMSE
    + MAE
    + MAPE
6. Discuss conclusions from your exercise to include: 
    + What are viable use cases and take-aways
    + What are potentially problematic conclusions one shouldn't take away
7. Submit your work for review via a sharable link to your Colab Codebook.

## Load Necessary Libraries

In [43]:
# Load necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

## Load Data into Dataframe

In [44]:
# Clone into the course github repository
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

fatal: destination path 'MSDIA_PredictiveModelingAndMachineLearning' already exists and is not an empty directory.


In [45]:
# Load data from csv file in the repository into wages dataframe if coming from github repo.
wages_df = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_II_9_Wages_1985_Current_Population_Survey.csv')
wages_df.head()

,Yrs_Ed,Sthrn_Rgn,Sex,Yrs_Exprnc,Union,Wage,Age,Race,Occup,Sect,Marr
0,8,0,1,21,0,5.10,35,H,Other,Manf,1
1,9,0,1,42,0,4.95,57,W,Other,Manf,1
2,12,0,0,1,0,6.67,19,W,Other,Manf,0
3,12,0,0,4,0,4.00,22,W,Other,Other,0
4,12,0,0,17,0,7.50,35,W,Other,Other,1


In [46]:
# Load data from csv file into wages dataframe if coming from local drive.
#wages_df = pd.read_csv('../Data/GB886_II_9_Wages_1985_Current_Population_Survey.csv')
#wages_df.head()

In [47]:
# Generate descriptive statistics for the wages dataframe
wages_df.describe()

,Yrs_Ed,Sthrn_Rgn,Sex,Yrs_Exprnc,Union,Wage,Age,Marr
count,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,13.018727,0.292135,0.458801,17.822097,0.179775,9.024064,36.833333,0.655431
std,2.615373,0.455170,0.498767,12.379710,0.384360,5.139097,11.726573,0.475673
min,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,18.000000,0.000000
25%,12.000000,0.000000,0.000000,8.000000,0.000000,5.250000,28.000000,0.000000
50%,12.000000,0.000000,0.000000,15.000000,0.000000,7.780000,35.000000,1.000000
75%,15.000000,1.000000,1.000000,26.000000,0.000000,11.250000,44.000000,1.000000
max,18.000000,1.000000,1.000000,55.000000,1.000000,44.500000,64.000000,1.000000


In [48]:
# Review the data in the Race column
wages_df['Race'].value_counts()

Race
W    440
O     67
H     27
Name: count, dtype: int64

In [49]:
# Review the data in the Occup column
wages_df['Occup'].value_counts()

Occup
Other       156
Prof        105
Clerical     97
Service      83
Mngmnt       55
Sales        38
Name: count, dtype: int64

In [50]:
# Review the data in the Sect column
wages_df['Sect'].value_counts()

Sect
Other    411
Manf      99
Con       24
Name: count, dtype: int64

## Prepare Data for Model

In [51]:
# Assign numerical columns to a list and categorical columns to another list.  Remove the Sthrn_Rgn, Sex, Union, and Marr columns from numerical and append these to categorical.
# Remove the wages column to be used as the target variable.
numerics = list(wages_df.select_dtypes(include=['int64']).columns)
numerics.remove('Sthrn_Rgn')
numerics.remove('Sex')
numerics.remove('Union')
numerics.remove('Marr')
factors = list(wages_df.select_dtypes(include=['object']).columns)
factors.append('Sthrn_Rgn')
factors.append('Sex')
factors.append('Union')
factors.append('Marr')


In [52]:
# Review the numerical columns
numeric_cols = wages_df[numerics]
numeric_cols.head()

,Yrs_Ed,Yrs_Exprnc,Age
0,8,21,35
1,9,42,57
2,12,1,19
3,12,4,22
4,12,17,35


In [53]:
# Review the categorical columns
factor_cols = wages_df[factors]
factor_cols.head()

,Race,Occup,Sect,Sthrn_Rgn,Sex,Union,Marr
0,H,Other,Manf,0,1,0,1
1,W,Other,Manf,0,1,0,1
2,W,Other,Manf,0,0,0,0
3,W,Other,Other,0,0,0,0
4,W,Other,Other,0,0,0,1


In [54]:
# Assign dummy values to the categorical columns
dummies = pd.get_dummies(factor_cols.astype('object'), drop_first=True)
factor_cols

,Race,Occup,Sect,Sthrn_Rgn,Sex,Union,Marr
0,H,Other,Manf,0,1,0,1
1,W,Other,Manf,0,1,0,1
2,W,Other,Manf,0,0,0,0
3,W,Other,Other,0,0,0,0
4,W,Other,Other,0,0,0,1
...,...,...,...,...,...,...,...
529,W,Prof,Other,0,0,0,0
530,O,Prof,Other,0,1,0,1
531,O,Prof,Other,0,1,1,1
532,W,Prof,Other,1,0,1,1


In [55]:
# Review the dummy variables
dummies

,Race_O,Race_W,Occup_Mngmnt,Occup_Other,Occup_Prof,Occup_Sales,Occup_Service,Sect_Manf,Sect_Other,Sthrn_Rgn_1,Sex_1,Union_1,Marr_1
0,False,False,False,True,False,False,False,True,False,False,True,False,True
1,False,True,False,True,False,False,False,True,False,False,True,False,True
2,False,True,False,True,False,False,False,True,False,False,False,False,False
3,False,True,False,True,False,False,False,False,True,False,False,False,False
4,False,True,False,True,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,False,True,False,False,True,False,False,False,True,False,False,False,False
530,True,False,False,False,True,False,False,False,True,False,True,False,True
531,True,False,False,False,True,False,False,False,True,False,True,True,True
532,False,True,False,False,True,False,False,False,True,True,False,True,True


In [56]:
# Combine the numerical and the dummy variables into a new dataframe
wages_cleaned = pd.concat([numeric_cols, dummies], axis=1)
wages_cleaned = pd.concat([wages_cleaned, wages_df['Wage']], axis=1)
wages_cleaned.head()

,Yrs_Ed,Yrs_Exprnc,Age,Race_O,Race_W,Occup_Mngmnt,Occup_Other,Occup_Prof,Occup_Sales,Occup_Service,Sect_Manf,Sect_Other,Sthrn_Rgn_1,Sex_1,Union_1,Marr_1,Wage
0,8,21,35,False,False,False,True,False,False,False,True,False,False,True,False,True,5.10
1,9,42,57,False,True,False,True,False,False,False,True,False,False,True,False,True,4.95
2,12,1,19,False,True,False,True,False,False,False,True,False,False,False,False,False,6.67
3,12,4,22,False,True,False,True,False,False,False,False,True,False,False,False,False,4.00
4,12,17,35,False,True,False,True,False,False,False,False,True,False,False,False,True,7.50


## Run Linear Regression Model

### Train Linear Regression Model

In [57]:
# Train the model using Ordinary Least Squares regression
X = wages_cleaned.drop('Wage', axis=1)
y = wages_cleaned['Wage']
X = sm.add_constant(X)  # Adds a constant term to the predictors
model = sm.OLS(y, X.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Wage   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     15.66
Date:                Sat, 04 Oct 2025   Prob (F-statistic):           3.02e-35
Time:                        16:37:41   Log-Likelihood:                -1525.8
No. Observations:                 534   AIC:                             3086.
Df Residuals:                     517   BIC:                             3158.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2814      6.741     -0.042      0.967     -13.525      12.962
Yrs_Ed            0.8128      1.087      0.748      0.455      -1.322       2.948
Yrs_Exprnc        0.2448      1.082      0.226      0.821      -1.880       2.370
Age              -0.1580      1.081     -0.146      0.884      -2.282       1.965
Race_O           -0.2314      0.992     -0.233      0.816      -2.179       1.717
Race_W            0.6065      0.870      0.697      0.486      -1.102       2.315
Occup_Mngmnt      3.2682      0.763      4.286      0.000       1.770       4.766
Occup_Other      -0.0223      0.686     -0.033      0.974      -1.370       1.325
Occup_Prof        1.9346      0.670      2.886      0.004       0.617       3.252
Occup_Sales      -0.7956      0.838     -0.949      0.343      -2.442       0.851
Occup_Service    -0.7072      0.663     -1.066      0.287      -2.010       0.596
Sect_Manf         0.5635      0.992      0.568      0.570      -1.384       2.511
Sect_Other       -0.4774      0.966     -0.494      0.621      -2.375       1.421
Sthrn_Rgn_1      -0.5627      0.420     -1.340      0.181      -1.388       0.262
Sex_1            -1.9425      0.419     -4.631      0.000      -2.767      -1.119
Union_1           1.6017      0.513      3.124      0.002       0.594       2.609
Marr_1            0.3005      0.411      0.731      0.465      -0.507       1.108
==============================================================================
Omnibus:                      244.027   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2320.734
Skew:                           1.756   Prob(JB):                         0.00
Kurtosis:                      12.590   Cond. No.                     1.72e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.72e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Calculate Required Metrics

In [58]:
# Report the R-sqaured value from the model summary above.
# The R-squared value is 0.364, indicating that approximately 36.4% of the variance in wages can be explained by the model.

In [59]:
# Calculate the RMSE
predictions = model.predict(X.astype(float))
rmse = np.sqrt(np.mean((y - predictions) ** 2))
print(f'Root Mean Squared Error: {round(rmse, 3)}')

Root Mean Squared Error: 4.214


In [60]:
# Calculate the MAE.
mae = np.mean(np.abs(y - predictions))
print(f'Mean Absolute Error: {round(mae, 3)}')

Mean Absolute Error: 2.988


In [61]:
# Calculate the MAPE
mape = np.mean(np.abs((y - predictions) / y)) * 100
print(f'Mean Absolute Percentage Error: {round(mape, 3)}%')

Mean Absolute Percentage Error: 40.687%


### Discuss Findings and Conclusions

#### Viable Use Cases
+ **Wage Determination Study**
    + Evaluating how education, experience, occupation, region, and union membership could influence wages.
    + Comparing relative pay differences based on certain features. (e.g., male vs. female, white vs. non-white, union vs non-union, etc.).
+ **Organizational Benchmarking**
    + Evaluating and providing estimates on how an additional year of education causes an increase in hourly wage. Assuming the model holds, the `Yrs_Ed` coef (~ 0.81) could be translated into an increase of $0.81/hour for every additional year of education.
    + Diving into if unionization does lead to a higher pay. Looking at the `Union_1` Coef (~1.60) does suggest that being unionized could lead to a wage difference of $1.60/hr over non-unionized workers.

#### Key Take-aways
+ **Main Influences on Wage**
    + Occupation is the most significant predictor: managerial and professional roles are strongly positive (p < 0.01).
    + Sex shows a large negative coefficient (-1.9425), which would suggest that in 1985 there was a substantial gender wage gap.
+ **Weak Predictors**
    + Surprisingly education, experience, and age are not significant influencers. This is probably due to other features having a larger impact on wage.
    + Region, maritial status, and industry(section) show essentially no impact on wage once occupation and union status are taken into account.

### Problematic Conclusions to Avoid
+ **Causality vs. Correlation**
    + As always the regression analysis captures association not casual impact. Therefore, causality does not mean correlation. A good example is unionization. Joining the union would not have meant an automatic increase of $1.60/hr in wage; other factors could be leading to that such as a higher paying sector could also be more unionized.
+ **Omitted-Variable Bias**
    + This analysis leaves out other factors that could have a large impact on wage, such as hours worked, firm size, location within a region, performance, etc. This could be the cause for the bias in coefficients for education and experience.